In [10]:
import material_interface as mi

ni_poscar = """Ni - primitive
1.0
   2.130422000	   0.000000000	   1.230000000
   0.710140800	   2.008582000	   1.230000000
   0.000000000	   0.000000000	   2.460000000
Ni
1
Selective dynamics
direct
   0.000000000    0.000000000    0.000000000 T T T Ni"""

graphene_poscar = """Graphene
1.0
   2.467291000	   0.000000000	   0.000000000
  -1.233645000	   2.136737000	   0.000000000
   0.000000000	   0.000000000	   7.803074000
C
2
direct
   0.000000000    0.000000000    0.000000000  C
   0.333333000    0.666667000    0.000000000  C
"""

from ase.build import surface, supercells
from ase.io import read, write
from ase.visualize import view
import io
import numpy as np

ni = mi.poscar_to_atoms(ni_poscar)
gr = mi.poscar_to_atoms(graphene_poscar)

print(ni.get_cell())
print(gr.get_cell())

ni_surface = surface(ni, (1, 1, 1), 1)
ni_slab = supercells.make_supercell(ni_surface, [[3, 0, 0], [0, 3, 0], [0, 0, 1]])
ni_slab.center(vacuum=10, axis=2)
ni_slab.wrap()

view(ni_slab + gr)

print(mi.write_atoms_to_poscar(ni))

Cell([[2.130422, 0.0, 1.23], [0.7101408, 2.008582, 1.23], [0.0, 0.0, 2.46]])
Cell([[2.467291, 0.0, 0.0], [-1.233645, 2.136737, 0.0], [0.0, 0.0, 7.803074]])
Ni 
 1.0000000000000000
     2.1304219999999998    0.0000000000000000    1.2300000000000000
     0.7101408000000000    2.0085820000000001    1.2300000000000000
     0.0000000000000000    0.0000000000000000    2.4600000000000000
 Ni 
   1
Cartesian
  0.0000000000000000  0.0000000000000000  0.0000000000000000



In [11]:
# !pip install git+https://github.com/nmdl-mizo/interface_master.git@develop

In [13]:
from interfacemaster.cellcalc import get_primitive_hkl, get_pri_vec_inplane, get_normal_index, get_normal_from_MI, rot
from interfacemaster.interface_generator import core, convert_vector_index
from interfacemaster.symmetric_tilt import get_csl_twisted_graphenes
from numpy import array, dot, round, cross, eye, pi, ceil, column_stack, arange
from numpy.linalg import inv, det, norm
from gb_code.csl_generator import print_list
import pandas as pd

ni_cif = mi.write_atoms_to_cif(ni_slab)
gr_cif = mi.write_atoms_to_cif(gr)

interface = core(ni_cif, gr_cif)

hkl_1 = get_primitive_hkl([0, 0, 1], interface.conv_lattice_1, interface.lattice_1)
hkl_2 = get_primitive_hkl([0, 0, 1], interface.conv_lattice_2, interface.lattice_2)

interface.parse_limit(du=1e-2, S=1e-2, sgm1=50, sgm2=50, dd=1e-2)

interface.search_one_position_2D(hkl_1, hkl_2, theta_range=10, dtheta=0.01, exact=False, start=10.89)

TypeError: string argument expected, got 'bytes'